In [117]:
from lusidtools.jupyter_tools import toggle_code

"""Total Return Swap Pricing With Reference Instrument

Demonstrates pricing of an Total Return Swap with AssetLeg having the ReferenceInstrument as underlying.

Attributes
----------
instruments
valuation
reference-instrument
"""

toggle_code("Hide docstring")

## Setup LUSID and LUSID API objects.

In [118]:
import os
from datetime import datetime, timedelta
import lusid
import json
import pandas as pd
import pytz
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken


# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)



LUSID Environment Initialised
LUSID SDK Version:  0.6.12724.0


In [119]:
# Setup the scope we'll use in this notebook:
scope = "trs-pricing-scope"
code = "trs-pricing-code"

coupon_rate = 0.015
start_date = datetime(2016, 9, 21, tzinfo=pytz.utc)
maturity_date = datetime(2047, 7, 22, tzinfo=pytz.utc)
dom_ccy = "GBP"
face_value = 1000

trade_date = datetime(2020, 6, 22, tzinfo=pytz.utc)
effective_at = datetime(2020, 6, 23, tzinfo=pytz.utc)

## Creating portfolio
We start by creating a portfolio which will consist of TotalReturnSwap only.

In [120]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lusid.CreateTransactionPortfolioRequest(
            display_name=code,
            code=code,
            base_currency=dom_ccy,
            created=start_date,
            instrument_scopes=[scope]
        ),
    )

except lusid.ApiException as e:
    if json.loads(e.body)['name'] == "PortfolioWithIdAlreadyExists":
        print("Portfolio with this id already exists.")
    else:
        raise e

Portfolio with this id already exists.


## Creating Instruments
A TotalReturnSwap is mainly composed of 2 parts, the AssetLeg and the FundingLeg. The AssetLeg is composed of direction (Pay/Receive) and the underlying instrument, that instrument could be inline definition of a Bond or ComplexBond or a ReferenceInstrument referencing already upserted Bond or ComplexBond. The second part is the FundingLeg which has to be either FixedLeg instrument or FloatLeg instrument inline.

One can insert a definition of a Bond or ComplexBond into TotalReturnSwap's AssetLeg directly without upserting it first, this is fairly common across Lusid. Upserting a ReferenceInstrument instead that references already existing instrument is new however and this is what will be focused on here. To do this, we will first upsert a Bond and then create a ReferenceInstrument that will point to it.

In [121]:
# Create and upsert a bond
bond_definition = models.Bond(
    start_date=start_date.isoformat(),
    maturity_date=maturity_date.isoformat(),
    dom_ccy=dom_ccy,
    coupon_rate=coupon_rate,
    principal=face_value,
    flow_conventions=models.FlowConventions(
        # coupon payment currency
        currency="GBP",
        # semi-annual coupon payments
        payment_frequency= "6M",
        # using an Actual/365 day count convention (other options : Act360, ActAct, ...
        day_count_convention="Act365",
        # modified following rolling convention (other options : ModifiedPrevious, NoAdjustment, EndOfMonth,...)
        roll_convention="ModifiedFollowing",
        # no holiday calendar supplied
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    identifiers={},
    instrument_type="Bond"
)
bond_id = "gilt2047s"
bond_upsert = instruments_api.upsert_instruments(request_body={bond_id: models.InstrumentDefinition(
    # instrument display name
    name="gilt 1.5% 47s",
    # unique instrument identifier
    identifiers={"ClientInternal": models.InstrumentIdValue(bond_id)},
    # our gilt instrument definition
    definition=bond_definition
)})
bond_luid = bond_upsert.values[bond_id].lusid_instrument_id

# Create ReferenceInstrument that points to the already upserted Bond.
reference_instrument = lusid.ReferenceInstrument(instrument_id=bond_id, scope="default", instrument_id_type="ClientInternal", instrument_type="ReferenceInstrument")


In [122]:
# Create a FixedLeg for the TotalReturnSwap
flow_conv = lusid.FlowConventions(
    currency=dom_ccy,
    payment_frequency="6M",
    day_count_convention="Act365",
    roll_convention="ModifiedFollowing",
    payment_calendars=[],
    reset_calendars=[],
    settle_days=0,
    reset_days=0
)

fixed_leg_definition = lusid.LegDefinition(
    rate_or_spread=0.02,
    pay_receive="Receive",
    conventions=flow_conv,
    stub_type="ShortFront",
    notional_exchange_type="None"
)

fixed_leg = lusid.FixedLeg(
    start_date=start_date,
    maturity_date=maturity_date,
    notional=2*face_value,
    leg_definition=fixed_leg_definition,
    instrument_type="FixedLeg"
)

In [123]:
# Create and upsert TRS
trs_definition = lusid.TotalReturnSwap(
    start_date=start_date,
    maturity_date=maturity_date,
    asset_leg=lusid.AssetLeg(pay_receive="Pay", asset=reference_instrument),
    instrument_type="TotalReturnSwap",
    funding_leg=fixed_leg
)
trs_id = "gilt2047sTrs"
trs_upsert = instruments_api.upsert_instruments(
    {
        trs_id: models.InstrumentDefinition(
        # instrument display name
        name="MyTRS",
        # unique instrument identifier
        identifiers={"ClientInternal": models.InstrumentIdValue(trs_id)},
        # our gilt instrument definition
        definition=trs_definition
    )})

    

## Creating Transaction

In [124]:

txn = lusid.TransactionRequest(
    transaction_id= trs_id,
    type="Buy",
    instrument_identifiers={"Instrument/default/ClientInternal": trs_id},
    transaction_date=trade_date.isoformat(),
    settlement_date=(trade_date + timedelta(days = 2)).isoformat(),
    units=1,
    transaction_price=lusid.TransactionPrice(price=1,type="Price"),
    total_consideration=lusid.CurrencyAndAmount(amount=1,currency=dom_ccy),
    exchange_rate=1,
    transaction_currency=dom_ccy
)

upsert_txn = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                          code=code,
                                                          transaction_request=[txn])

## Creating ConfigurationRecipe

In [125]:
market_supplier = "Lusid"
recipe = models.ConfigurationRecipe(
    scope=scope,
    code=code,
    description="Price bond using discounting model",
    market=lusid.MarketContext(
        market_rules=[
            lusid.MarketDataKeyRule(
                key="Rates.*.*",
                supplier=market_supplier,
                data_scope="default",
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="100D",
            ),
        ],
        options=lusid.MarketOptions(
            default_scope = scope,
            attempt_to_infer_missing_fx=True
        ),
    ),
    pricing=models.PricingContext(
        # select the "Discounting" model for bond pricing
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="Bond",
                parameters="{}"
            ),
            models.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="FixedLeg",
                parameters="{}"
            ),
        ],
        options=models.PricingOptions(model_selection=models.ModelSelection(library="Lusid", model="ConstantTimeValueOfMoney"))
    )
)

# Upsert recipe to LUSID
upsert_recipe_request = models.UpsertRecipeRequest(configuration_recipe=recipe)
response = api_factory.build(lusid.api.ConfigurationRecipeApi).upsert_configuration_recipe(upsert_recipe_request)

## Performing valuation
Now that we have a portfolio with valid TotalReturnSwap we can perform a valuation on it. The ReferenceInstrument will be resolved automatically.

In [126]:
def run_valuation(date):
    metrics = [
        lusid.AggregateSpec("Instrument/default/Name", "Value"),
        lusid.AggregateSpec("Valuation/PV/Amount", "Value"),
        lusid.AggregateSpec("Valuation/InstrumentAccrued", "Value")
    ]

    group_by = []

    valuation_request = lusid.ValuationRequest(
        recipe_id=lusid.ResourceId(scope=scope, code=code),
        metrics=metrics,
        group_by=group_by,
        portfolio_entity_ids=[
            lusid.PortfolioEntityId(scope=scope, code=code)
        ],
        valuation_schedule=lusid.ValuationSchedule(effective_at=date),
    )

    val_data = aggregation_api.get_valuation(valuation_request=valuation_request).data
 
    vals_df = pd.DataFrame(val_data)

    vals_df.fillna("", inplace=True)

    return vals_df

pricing_date = trade_date + timedelta(weeks=52)
valuation = run_valuation(pricing_date.isoformat())
display(valuation.head(1))

,Valuation/PV/Amount,Instrument/default/Name,Valuation/InstrumentAccrued
0,-865.19863,MyTRS,2.054795
